In [3]:
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.pipelines import pipeline
import torch

load_dotenv()
model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id, return_tensorts='pt', add_special_tokens=False)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='cuda', torch_dtype = torch.float16)
llama_pipeline = pipeline('text-generation', model = model, tokenizer=tokenizer)


loading file tokenizer.model from cache at /home/sos00/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf/snapshots/92011f62d7604e261f748ec0cfe6329f31193e33/tokenizer.model
loading file tokenizer.json from cache at /home/sos00/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf/snapshots/92011f62d7604e261f748ec0cfe6329f31193e33/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/sos00/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf/snapshots/92011f62d7604e261f748ec0cfe6329f31193e33/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/sos00/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf/snapshots/92011f62d7604e261f748ec0cfe6329f31193e33/tokenizer_config.json
loading configuration file config.json from cache at /home/sos00/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf/snapshots/92011f62d7604e261f748ec0cfe6329f31193e33/config.jso

In [4]:
!nvidia-smi

Thu Apr 11 09:28:31 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.06              Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8             13W /  124W |    7976MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
llama_pipeline('How can I learn to speak mandarin?', max_length=100)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'How can I learn to speak mandarin?\n\nAnswer: There are many ways to learn Mandarin Chinese, including:\n\n1. Enrolling in a class: Look for local colleges, universities, or language schools that offer Mandarin classes. This will provide you with a structured learning environment and a teacher to guide you.\n2. Hiring a tutor: You can find a private tutor who will give you one-on-one lessons and tailor'}]

In [6]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=llama_pipeline)

In [9]:
llm.invoke('How can I learn chinese?', pipeline_kwargs={"max_new_tokens": 20})

'How can I learn chinese?\n\nAnswer: There are many ways to learn Chinese, and the best method for you will depend'

In [17]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', "You are a useful and attentive assitant. Please respond to user queries. Please keep your answer very concise."),
        ("human", "Question:{question}"),
    ]
)
chain = prompt | llm
chain.invoke({'question':'What goes well with ketchup?'})

TypeError: RunnableSequence.invoke() got an unexpected keyword argument 'pipeline_kwargs'